## Final Project - Data Science Capstone

By Sara Elgayar

Data Science Masters student at University of Rochester

### IMPORT LIBRARIES

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.feature_selection import SelectKBest
import warnings
warnings.filterwarnings('ignore')

## LOAD DATASET

In [2]:
items = pd.read_csv('2022 Data/items.csv')
transactions = pd.read_csv('2022 Data/transactions-locationnbr-40.csv')
dates = pd.read_csv('2022 Data/dates.csv')
monthly_sales = pd.read_csv('2022 Data/aggregatedsales-by-month-locationnbr-40.csv')
pricing = pd.read_csv('2022 Data/pricinghistory-by-month-locationnbr-40.csv')

In [3]:
items.head()

(155154, 18)

## DATA CLEANING


1. I filtered agg sales for >0 Ecom units sold (so we’re not 
 including cases where something was bought and returned) 

2. and filtered >0 ecom dollars sold (so we’re not including any losses because of coupons) 

3. removed any NANs
    
4. removed any BRAND_NAME = '[Not Set]'

5. Same for Total units and dollars sold


In [4]:
# remove Ecom units <0 
monthly_sales = monthly_sales[monthly_sales.ECOMM_UNITS_SOLD > 0]
# remove ecom dollars sold <0 
monthly_sales = monthly_sales[monthly_sales.ECOMM_DOLLARS_SOLD > 0]
# remove Total units <0 
monthly_sales = monthly_sales[monthly_sales.TOTAL_UNITS_SOLD > 0]
# remove Total dollars sold <0 
monthly_sales = monthly_sales[monthly_sales.TOTAL_DOLLARS_SOLD > 0]
# delete BRAND_NAME = '[Not Set]'
items = items[items.BRAND_NAME != '[Not Set]']
items.shape

(122913, 18)

Remove NANs

In [5]:
# check for null values
# items.isnull().values.sum() # 15

# check for null values
# pricing.isnull().values.sum() # 100

In [6]:
items = items.dropna()
pricing = pricing.dropna()
items.shape

(122898, 18)

## MERGE DATA FROM DIFFERENT SOURCES

In [7]:
training_df = pd.read_csv('training_data_2022_Q3.csv')

In [8]:
transactions_date =  pd.merge(transactions, dates, how = 'inner')

# split transactions by quarters
# Q3 = January, February, and March
# Q3 = April, May and June
# Q3 = July, August, and September
# Q4 = October, November, and December

transactions_date_2022_Q3 = transactions_date[(transactions_date['FSCL_YEAR_NBR'] == 2022) & (transactions_date['FSCL_MONTH_NBR'] <=9)  & (transactions_date['FSCL_MONTH_NBR'] >= 7)]

# Merge transactions and pricing data
transactions_date_2022_Q3 =  pd.merge(transactions_date_2022_Q3, pricing, on='ITEM_NBR')

# Split monthly sales by quarters
monthly_sales_2022_Q3 = monthly_sales[(monthly_sales['FSCL_YEAR_NBR'] == 2022) &  (monthly_sales['FSCL_MONTH_NBR'] <= 9)  & (monthly_sales['FSCL_MONTH_NBR'] >= 7)]



In [9]:
monthly_sales_2022_Q3.shape

(40576, 9)

## TRANING DATASET CREATION

Add 'ITEM_NBR', 'ORGANIC_IND' features for each item

In [10]:
# Add BRAND_NAME to get wegmans brand or national brand
training_data_2022_Q3 = items[['ITEM_NBR', 'ORGANIC_IND', 'BRAND_NAME', 'DEPARTMENT_NAME', 'CATEGORY_NAME', 'CLASS_NAME']]
training_data_2022_Q3.shape

(122898, 6)

### 1. Add 'ONLINE_UNITS_SOLD_2022' feature for each item

In [11]:
# Get Ecomm units sold in 2022 Q3, if item is not included in 2022 Q3 then ECOMM_UNITS_SOLD = 0
monthly_units_sold_2022_Q3 = pd.DataFrame(monthly_sales_2022_Q3.groupby(['ITEM_NBR'])['ECOMM_UNITS_SOLD'].sum().items(), columns=['ITEM_NBR', 'ONLINE_UNITS_SOLD_2022_Q3'])


In [12]:
monthly_sales_2022_Q3.ITEM_NBR.nunique()

18670

In [13]:
training_data_2022_Q3 = pd.merge(training_data_2022_Q3, monthly_units_sold_2022_Q3[['ITEM_NBR', 'ONLINE_UNITS_SOLD_2022_Q3']] , on ='ITEM_NBR', how = 'left')


In [14]:
# if item has no ONLINE_UNITS_SOLD_2022_Q3, then set ONLINE_UNITS_SOLD_2022_Q3 = 0
training_data_2022_Q3['ONLINE_UNITS_SOLD_2022_Q3'] = training_data_2022_Q3['ONLINE_UNITS_SOLD_2022_Q3'].replace(np.nan, 0)
training_data_2022_Q3.shape

(122898, 7)

In [15]:
training_data_2022_Q3.shape

(122898, 7)

## 2. Add 'AGGREGATE_UNITS_SOLD_2022_Q3' feature for each item

In [16]:
# Get Total units sold in 2022 Q3, if item is not included in 2022 Q3 then AGGREGATE_UNITS_SOLD_2022_Q3 = 0
monthly_tot_units_sold = pd.DataFrame(monthly_sales_2022_Q3.groupby(['ITEM_NBR'])['TOTAL_UNITS_SOLD'].sum().items(), columns=['ITEM_NBR', 'AGGREGATE_UNITS_SOLD_2022_Q3'])
training_data_2022_Q3 = pd.merge(training_data_2022_Q3, monthly_tot_units_sold[['ITEM_NBR', 'AGGREGATE_UNITS_SOLD_2022_Q3']] , on ='ITEM_NBR', how='left')


In [17]:
monthly_tot_units_sold

,ITEM_NBR,AGGREGATE_UNITS_SOLD_2022_Q3
0,205.0,477
1,227.0,763
2,228.0,3505
3,229.0,720
4,257.0,200
...,...,...
18665,891829.0,3
18666,891902.0,18
18667,892440.0,9
18668,892933.0,32


In [18]:
# if item has no AGGREGATE_UNITS_SOLD_2022_Q3, then set AGGREGATE_UNITS_SOLD_2022_Q3 = 0
training_data_2022_Q3['AGGREGATE_UNITS_SOLD_2022_Q3'] = training_data_2022_Q3['AGGREGATE_UNITS_SOLD_2022_Q3'].replace(np.nan, 0)
training_data_2022_Q3

,ITEM_NBR,ORGANIC_IND,BRAND_NAME,DEPARTMENT_NAME,CATEGORY_NAME,CLASS_NAME,ONLINE_UNITS_SOLD_2022_Q3,AGGREGATE_UNITS_SOLD_2022_Q3
0,47450.0,0,Wegmans,PRODUCE,JUICE AND CIDER,JUICE,6.0,618.0
1,49742.0,1,Wegmans,PRODUCE,JUICE AND CIDER,ORGANIC JUICE,4.0,154.0
2,51493.0,0,Wegmans,PRODUCE,SALADS AND SALAD KITS,SALAD BOWLS,73.0,907.0
3,43111.0,0,Wegmans,PRODUCE,TROPICAL,EXOTICS,0.0,0.0
4,85223.0,0,Pepperidge Farm,BAKESHOP BREAD & ROLLS,VENDOR ROLLS,PEPPERIDGE FARM ROLLS,20.0,573.0
...,...,...,...,...,...,...,...,...
122893,452470.0,0,Wegmans,FLORAL,SPECIAL ORDERS,ROSES,0.0,0.0
122894,441105.0,0,Wegmans,FLORAL,FRESH BUNCHES,GB FOCAL FLOWERS,0.0,0.0
122895,286569.0,0,Wegmans,FLORAL,PLANTS,TO BE DELETED,0.0,0.0
122896,226396.0,0,Wegmans,FLORAL,ICE WHOLESALER CODES,ICE WHOLESALER CODES,0.0,0.0


### 3. Add 'DEPARTMENT_SALES_PERC' feature

In [19]:
monthly_items = pd.merge(monthly_sales_2022_Q3, items, on = 'ITEM_NBR')

In [20]:
dep_perc = pd.DataFrame(monthly_items[['DEPARTMENT_NAME']].value_counts(normalize=True).reset_index())
dep_perc.columns = ['DEPARTMENT_NAME', 'DEPARTMENT_SALES_PERC']

In [21]:
dep_perc.head(20)

,DEPARTMENT_NAME,DEPARTMENT_SALES_PERC
0,GROCERY,0.523943
1,FROZEN FOOD,0.099595
2,HEALTH & BEAUTY CARE,0.079314
3,DAIRY,0.069533
4,PRODUCE,0.044684
5,ICE CREAM,0.032867
6,CULTURED DAIRY,0.029615
7,MEAT,0.023012
8,OLDE WORLD CHEESE,0.012065
9,GM BASIC PRODUCTS DEPARTMENT,0.009756


In [22]:
training_data_2022_Q3 = pd.merge(training_data_2022_Q3, dep_perc, on='DEPARTMENT_NAME', how='left')
training_data_2022_Q3

,ITEM_NBR,ORGANIC_IND,BRAND_NAME,DEPARTMENT_NAME,CATEGORY_NAME,CLASS_NAME,ONLINE_UNITS_SOLD_2022_Q3,AGGREGATE_UNITS_SOLD_2022_Q3,DEPARTMENT_SALES_PERC
0,47450.0,0,Wegmans,PRODUCE,JUICE AND CIDER,JUICE,6.0,618.0,0.044684
1,49742.0,1,Wegmans,PRODUCE,JUICE AND CIDER,ORGANIC JUICE,4.0,154.0,0.044684
2,51493.0,0,Wegmans,PRODUCE,SALADS AND SALAD KITS,SALAD BOWLS,73.0,907.0,0.044684
3,43111.0,0,Wegmans,PRODUCE,TROPICAL,EXOTICS,0.0,0.0,0.044684
4,85223.0,0,Pepperidge Farm,BAKESHOP BREAD & ROLLS,VENDOR ROLLS,PEPPERIDGE FARM ROLLS,20.0,573.0,0.009557
...,...,...,...,...,...,...,...,...,...
122893,452470.0,0,Wegmans,FLORAL,SPECIAL ORDERS,ROSES,0.0,0.0,0.002284
122894,441105.0,0,Wegmans,FLORAL,FRESH BUNCHES,GB FOCAL FLOWERS,0.0,0.0,0.002284
122895,286569.0,0,Wegmans,FLORAL,PLANTS,TO BE DELETED,0.0,0.0,0.002284
122896,226396.0,0,Wegmans,FLORAL,ICE WHOLESALER CODES,ICE WHOLESALER CODES,0.0,0.0,0.002284


In [23]:
# if item has no AGGREGATE_UNITS_SOLD_2022_Q3, then set AGGREGATE_UNITS_SOLD_2022_Q3 = 0
training_data_2022_Q3['DEPARTMENT_SALES_PERC'] = training_data_2022_Q3['DEPARTMENT_SALES_PERC'].replace(np.nan, 0)
training_data_2022_Q3

,ITEM_NBR,ORGANIC_IND,BRAND_NAME,DEPARTMENT_NAME,CATEGORY_NAME,CLASS_NAME,ONLINE_UNITS_SOLD_2022_Q3,AGGREGATE_UNITS_SOLD_2022_Q3,DEPARTMENT_SALES_PERC
0,47450.0,0,Wegmans,PRODUCE,JUICE AND CIDER,JUICE,6.0,618.0,0.044684
1,49742.0,1,Wegmans,PRODUCE,JUICE AND CIDER,ORGANIC JUICE,4.0,154.0,0.044684
2,51493.0,0,Wegmans,PRODUCE,SALADS AND SALAD KITS,SALAD BOWLS,73.0,907.0,0.044684
3,43111.0,0,Wegmans,PRODUCE,TROPICAL,EXOTICS,0.0,0.0,0.044684
4,85223.0,0,Pepperidge Farm,BAKESHOP BREAD & ROLLS,VENDOR ROLLS,PEPPERIDGE FARM ROLLS,20.0,573.0,0.009557
...,...,...,...,...,...,...,...,...,...
122893,452470.0,0,Wegmans,FLORAL,SPECIAL ORDERS,ROSES,0.0,0.0,0.002284
122894,441105.0,0,Wegmans,FLORAL,FRESH BUNCHES,GB FOCAL FLOWERS,0.0,0.0,0.002284
122895,286569.0,0,Wegmans,FLORAL,PLANTS,TO BE DELETED,0.0,0.0,0.002284
122896,226396.0,0,Wegmans,FLORAL,ICE WHOLESALER CODES,ICE WHOLESALER CODES,0.0,0.0,0.002284


### 4. Add 'ITEM_SALES_RANK_IN_DEPARTMENT' feature

In [24]:
# Merge monthly_sales with items to get the ITEM_SALES_RANK_IN_DEPARTMENT for each item
sales_items = pd.merge(monthly_sales_2022_Q3, items, on='ITEM_NBR')

In [25]:
ITEM_TOTAL_UNITS_SOLD_df = sales_items.groupby('ITEM_NBR')['TOTAL_UNITS_SOLD'].sum().reset_index().rename(columns={'TOTAL_UNITS_SOLD':'ITEM_TOTAL_UNITS_SOLD'})
ITEM_TOTAL_UNITS_SOLD_df.head()

,ITEM_NBR,ITEM_TOTAL_UNITS_SOLD
0,205.0,477
1,227.0,763
2,228.0,3505
3,229.0,720
4,257.0,200


In [26]:
sales_items = pd.merge(sales_items, ITEM_TOTAL_UNITS_SOLD_df, on='ITEM_NBR')
sales_items.head()

,FSCL_YEAR_NBR,FSCL_MONTH_NBR,FSCL_MONTH_NAME,ITEM_NBR,LOCATION_NBR,TOTAL_UNITS_SOLD,TOTAL_DOLLARS_SOLD,ECOMM_UNITS_SOLD,ECOMM_DOLLARS_SOLD,ITEM_DESC_SHORT,...,DEPARTMENT_NAME,CATEGORY_FULL_NBR,CATEGORY_NAME,CLASS_FULL_NBR,CLASS_NAME,ITEM_SIZE,ITEM_UOM_DESC,ITEM_STATUS_DESC,IS_CURRENT_WOODMORE_FASTPICK_FLG,ITEM_TOTAL_UNITS_SOLD
0,2022,7,July,205.0,40,160,960.00,5.0,34.45,7 Grain,...,IN-STORE BREAD & ROLLS,54004,CORE BREADS,54004005,7 GRAIN,20.0,OZ,Store Active,0,477
1,2022,8,August,205.0,40,158,948.00,6.0,41.34,7 Grain,...,IN-STORE BREAD & ROLLS,54004,CORE BREADS,54004005,7 GRAIN,20.0,OZ,Store Active,0,477
2,2022,9,September,205.0,40,159,954.00,3.0,21.27,7 Grain,...,IN-STORE BREAD & ROLLS,54004,CORE BREADS,54004005,7 GRAIN,20.0,OZ,Store Active,0,477
3,2022,7,July,227.0,40,273,1171.17,5.0,24.95,WB Micro Spinach,...,PRODUCE,3013,SALADS AND SALAD KITS,3013020,PACKAGED COOKING GREENS,9.0,OZ,Store Active,0,763
4,2022,8,August,227.0,40,284,1218.36,7.0,34.93,WB Micro Spinach,...,PRODUCE,3013,SALADS AND SALAD KITS,3013020,PACKAGED COOKING GREENS,9.0,OZ,Store Active,0,763


In [27]:
# compute ITEM_SALES_RANK_IN_DEPARTMENT
sales_items['ITEM_SALES_RANK_IN_DEPARTMENT'] = sales_items['ITEM_TOTAL_UNITS_SOLD'].rank(method='dense',pct=True, ascending=False)


In [28]:
# sort dept rank
sales_items = sales_items.sort_values(['DEPARTMENT_NAME', 'ITEM_SALES_RANK_IN_DEPARTMENT'])

# merge dept rank and training data
training_data_2022_Q3 = pd.merge(training_data_2022_Q3, sales_items[['ITEM_NBR', 'ITEM_SALES_RANK_IN_DEPARTMENT']] , on ='ITEM_NBR', how='left')
training_data_2022_Q3


,ITEM_NBR,ORGANIC_IND,BRAND_NAME,DEPARTMENT_NAME,CATEGORY_NAME,CLASS_NAME,ONLINE_UNITS_SOLD_2022_Q3,AGGREGATE_UNITS_SOLD_2022_Q3,DEPARTMENT_SALES_PERC,ITEM_SALES_RANK_IN_DEPARTMENT
0,47450.0,0,Wegmans,PRODUCE,JUICE AND CIDER,JUICE,6.0,618.0,0.044684,0.597769
1,47450.0,0,Wegmans,PRODUCE,JUICE AND CIDER,JUICE,6.0,618.0,0.044684,0.597769
2,49742.0,1,Wegmans,PRODUCE,JUICE AND CIDER,ORGANIC JUICE,4.0,154.0,0.044684,0.899606
3,49742.0,1,Wegmans,PRODUCE,JUICE AND CIDER,ORGANIC JUICE,4.0,154.0,0.044684,0.899606
4,51493.0,0,Wegmans,PRODUCE,SALADS AND SALAD KITS,SALAD BOWLS,73.0,907.0,0.044684,0.444882
...,...,...,...,...,...,...,...,...,...,...
144659,452470.0,0,Wegmans,FLORAL,SPECIAL ORDERS,ROSES,0.0,0.0,0.002284,NaN
144660,441105.0,0,Wegmans,FLORAL,FRESH BUNCHES,GB FOCAL FLOWERS,0.0,0.0,0.002284,NaN
144661,286569.0,0,Wegmans,FLORAL,PLANTS,TO BE DELETED,0.0,0.0,0.002284,NaN
144662,226396.0,0,Wegmans,FLORAL,ICE WHOLESALER CODES,ICE WHOLESALER CODES,0.0,0.0,0.002284,NaN


In [29]:
# if an item has no DEPARTMENT_SALES_RANK, then set DEPARTMENT_SALES_RANK = 0
training_data_2022_Q3['ITEM_SALES_RANK_IN_DEPARTMENT'] = training_data_2022_Q3['ITEM_SALES_RANK_IN_DEPARTMENT'].replace(np.nan, 0)
training_data_2022_Q3


,ITEM_NBR,ORGANIC_IND,BRAND_NAME,DEPARTMENT_NAME,CATEGORY_NAME,CLASS_NAME,ONLINE_UNITS_SOLD_2022_Q3,AGGREGATE_UNITS_SOLD_2022_Q3,DEPARTMENT_SALES_PERC,ITEM_SALES_RANK_IN_DEPARTMENT
0,47450.0,0,Wegmans,PRODUCE,JUICE AND CIDER,JUICE,6.0,618.0,0.044684,0.597769
1,47450.0,0,Wegmans,PRODUCE,JUICE AND CIDER,JUICE,6.0,618.0,0.044684,0.597769
2,49742.0,1,Wegmans,PRODUCE,JUICE AND CIDER,ORGANIC JUICE,4.0,154.0,0.044684,0.899606
3,49742.0,1,Wegmans,PRODUCE,JUICE AND CIDER,ORGANIC JUICE,4.0,154.0,0.044684,0.899606
4,51493.0,0,Wegmans,PRODUCE,SALADS AND SALAD KITS,SALAD BOWLS,73.0,907.0,0.044684,0.444882
...,...,...,...,...,...,...,...,...,...,...
144659,452470.0,0,Wegmans,FLORAL,SPECIAL ORDERS,ROSES,0.0,0.0,0.002284,0.000000
144660,441105.0,0,Wegmans,FLORAL,FRESH BUNCHES,GB FOCAL FLOWERS,0.0,0.0,0.002284,0.000000
144661,286569.0,0,Wegmans,FLORAL,PLANTS,TO BE DELETED,0.0,0.0,0.002284,0.000000
144662,226396.0,0,Wegmans,FLORAL,ICE WHOLESALER CODES,ICE WHOLESALER CODES,0.0,0.0,0.002284,0.000000


### 5. Add 'CATEGORY_SALES_PERC' Feature

In [30]:
cat_perc = pd.DataFrame(monthly_items[['CATEGORY_NAME']].value_counts(normalize=True).reset_index())
cat_perc.columns = ['CATEGORY_NAME', 'CATEGORY_SALES_PERC']

In [31]:
cat_perc.head()

,CATEGORY_NAME,CATEGORY_SALES_PERC
0,CHIPS & SNACKS,0.032768
1,YOGURT,0.023161
2,SPICES & SEASONINGS,0.019388
3,CARBONATED SODA POP,0.019289
4,LATIN FOODS,0.019015


In [32]:
training_data_2022_Q3 = pd.merge(training_data_2022_Q3, cat_perc, on='CATEGORY_NAME', how='left')
training_data_2022_Q3.shape

(144664, 11)

In [33]:
# if an item has no DEPARTMENT_SALES_RANK, then set DEPARTMENT_SALES_RANK = 0
training_data_2022_Q3['CATEGORY_SALES_PERC'] = training_data_2022_Q3['CATEGORY_SALES_PERC'].replace(np.nan, 0)
training_data_2022_Q3

,ITEM_NBR,ORGANIC_IND,BRAND_NAME,DEPARTMENT_NAME,CATEGORY_NAME,CLASS_NAME,ONLINE_UNITS_SOLD_2022_Q3,AGGREGATE_UNITS_SOLD_2022_Q3,DEPARTMENT_SALES_PERC,ITEM_SALES_RANK_IN_DEPARTMENT,CATEGORY_SALES_PERC
0,47450.0,0,Wegmans,PRODUCE,JUICE AND CIDER,JUICE,6.0,618.0,0.044684,0.597769,0.001837
1,47450.0,0,Wegmans,PRODUCE,JUICE AND CIDER,JUICE,6.0,618.0,0.044684,0.597769,0.001837
2,49742.0,1,Wegmans,PRODUCE,JUICE AND CIDER,ORGANIC JUICE,4.0,154.0,0.044684,0.899606,0.001837
3,49742.0,1,Wegmans,PRODUCE,JUICE AND CIDER,ORGANIC JUICE,4.0,154.0,0.044684,0.899606,0.001837
4,51493.0,0,Wegmans,PRODUCE,SALADS AND SALAD KITS,SALAD BOWLS,73.0,907.0,0.044684,0.444882,0.006032
...,...,...,...,...,...,...,...,...,...,...,...
144659,452470.0,0,Wegmans,FLORAL,SPECIAL ORDERS,ROSES,0.0,0.0,0.002284,0.000000,0.000000
144660,441105.0,0,Wegmans,FLORAL,FRESH BUNCHES,GB FOCAL FLOWERS,0.0,0.0,0.002284,0.000000,0.000770
144661,286569.0,0,Wegmans,FLORAL,PLANTS,TO BE DELETED,0.0,0.0,0.002284,0.000000,0.000248
144662,226396.0,0,Wegmans,FLORAL,ICE WHOLESALER CODES,ICE WHOLESALER CODES,0.0,0.0,0.002284,0.000000,0.000000


### 6. Add 'CLASS_SALES_RANK' feature for each item 


In [34]:
class_perc = pd.DataFrame(monthly_items[['CLASS_NAME']].value_counts(normalize=True).reset_index())
class_perc.columns = ['CLASS_NAME', 'CLASS_SALES_PERC']

In [35]:
class_perc.head()

,CLASS_NAME,CLASS_SALES_PERC
0,GREEK - STRAINED,0.009036
1,POTATO CHIPS,0.007224
2,POURABLE SALAD DRES REG,0.006529
3,SNACK BARS,0.006454
4,DINNERS MID TIER,0.006430


In [36]:
training_data_2022_Q3 = pd.merge(training_data_2022_Q3, class_perc, on='CLASS_NAME', how='left')
training_data_2022_Q3.shape

(144664, 12)

In [37]:
# if an item has no DEPARTMENT_SALES_RANK, then set DEPARTMENT_SALES_RANK = 0
training_data_2022_Q3['CLASS_SALES_PERC'] = training_data_2022_Q3['CLASS_SALES_PERC'].replace(np.nan, 0)
training_data_2022_Q3

,ITEM_NBR,ORGANIC_IND,BRAND_NAME,DEPARTMENT_NAME,CATEGORY_NAME,CLASS_NAME,ONLINE_UNITS_SOLD_2022_Q3,AGGREGATE_UNITS_SOLD_2022_Q3,DEPARTMENT_SALES_PERC,ITEM_SALES_RANK_IN_DEPARTMENT,CATEGORY_SALES_PERC,CLASS_SALES_PERC
0,47450.0,0,Wegmans,PRODUCE,JUICE AND CIDER,JUICE,6.0,618.0,0.044684,0.597769,0.001837,0.000596
1,47450.0,0,Wegmans,PRODUCE,JUICE AND CIDER,JUICE,6.0,618.0,0.044684,0.597769,0.001837,0.000596
2,49742.0,1,Wegmans,PRODUCE,JUICE AND CIDER,ORGANIC JUICE,4.0,154.0,0.044684,0.899606,0.001837,0.000770
3,49742.0,1,Wegmans,PRODUCE,JUICE AND CIDER,ORGANIC JUICE,4.0,154.0,0.044684,0.899606,0.001837,0.000770
4,51493.0,0,Wegmans,PRODUCE,SALADS AND SALAD KITS,SALAD BOWLS,73.0,907.0,0.044684,0.444882,0.006032,0.000571
...,...,...,...,...,...,...,...,...,...,...,...,...
144659,452470.0,0,Wegmans,FLORAL,SPECIAL ORDERS,ROSES,0.0,0.0,0.002284,0.000000,0.000000,0.000000
144660,441105.0,0,Wegmans,FLORAL,FRESH BUNCHES,GB FOCAL FLOWERS,0.0,0.0,0.002284,0.000000,0.000770,0.000298
144661,286569.0,0,Wegmans,FLORAL,PLANTS,TO BE DELETED,0.0,0.0,0.002284,0.000000,0.000248,0.000000
144662,226396.0,0,Wegmans,FLORAL,ICE WHOLESALER CODES,ICE WHOLESALER CODES,0.0,0.0,0.002284,0.000000,0.000000,0.000000


### (7&8) Add 'Original_ORDERED_COUNT' & ''Substitution_ORDERED_COUNT'

In [38]:
# get number of times an item was sub ordered
sub_df = pd.DataFrame(transactions[transactions.FULFILLED_ITEM_TYPE == 'Substitution Item'].ITEM_NBR.value_counts().items(), columns=['ITEM_NBR', 'Substitution_ORDERED_COUNT'])
                                                                                            
# get number of times an item was org ordered
org_df = pd.DataFrame(transactions[transactions.FULFILLED_ITEM_TYPE == 'Original Item'].ITEM_NBR.value_counts().items(), columns=['ITEM_NBR', 'Original_ORDERED_COUNT'])


In [39]:
                                                                                            
# merge sub_df + training_data
training_data_2022_Q3 = pd.merge(training_data_2022_Q3, sub_df, on='ITEM_NBR', how='left')
# if an item has no SUBSTITUTION_ORDERED_COUNT, then set SUBSTITUTION_ORDERED_COUNT = 0
training_data_2022_Q3['Substitution_ORDERED_COUNT'] = training_data_2022_Q3['Substitution_ORDERED_COUNT'].replace(np.nan, 0)

# merge org_df + training_data
training_data_2022_Q3 = pd.merge(training_data_2022_Q3, org_df, on='ITEM_NBR', how='left')


# if an item has no ORIGINALLY_ORDERED_COUNT, then set ORIGINALLY_ORDERED_COUNT = 0
training_data_2022_Q3['Original_ORDERED_COUNT'] = training_data_2022_Q3['Original_ORDERED_COUNT'].replace(np.nan, 0)
training_data_2022_Q3


,ITEM_NBR,ORGANIC_IND,BRAND_NAME,DEPARTMENT_NAME,CATEGORY_NAME,CLASS_NAME,ONLINE_UNITS_SOLD_2022_Q3,AGGREGATE_UNITS_SOLD_2022_Q3,DEPARTMENT_SALES_PERC,ITEM_SALES_RANK_IN_DEPARTMENT,CATEGORY_SALES_PERC,CLASS_SALES_PERC,Substitution_ORDERED_COUNT,Original_ORDERED_COUNT
0,47450.0,0,Wegmans,PRODUCE,JUICE AND CIDER,JUICE,6.0,618.0,0.044684,0.597769,0.001837,0.000596,14.0,15.0
1,47450.0,0,Wegmans,PRODUCE,JUICE AND CIDER,JUICE,6.0,618.0,0.044684,0.597769,0.001837,0.000596,14.0,15.0
2,49742.0,1,Wegmans,PRODUCE,JUICE AND CIDER,ORGANIC JUICE,4.0,154.0,0.044684,0.899606,0.001837,0.000770,10.0,14.0
3,49742.0,1,Wegmans,PRODUCE,JUICE AND CIDER,ORGANIC JUICE,4.0,154.0,0.044684,0.899606,0.001837,0.000770,10.0,14.0
4,51493.0,0,Wegmans,PRODUCE,SALADS AND SALAD KITS,SALAD BOWLS,73.0,907.0,0.044684,0.444882,0.006032,0.000571,24.0,175.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144659,452470.0,0,Wegmans,FLORAL,SPECIAL ORDERS,ROSES,0.0,0.0,0.002284,0.000000,0.000000,0.000000,0.0,0.0
144660,441105.0,0,Wegmans,FLORAL,FRESH BUNCHES,GB FOCAL FLOWERS,0.0,0.0,0.002284,0.000000,0.000770,0.000298,0.0,0.0
144661,286569.0,0,Wegmans,FLORAL,PLANTS,TO BE DELETED,0.0,0.0,0.002284,0.000000,0.000248,0.000000,0.0,0.0
144662,226396.0,0,Wegmans,FLORAL,ICE WHOLESALER CODES,ICE WHOLESALER CODES,0.0,0.0,0.002284,0.000000,0.000000,0.000000,0.0,0.0


### 9. ADD 'STORE_BRAND_IND' Feature

In [40]:
# set STORE_BRAND_IND = 1, if an BRAND_NAME = Wegmans, else BRAND_NAME = 0
training_data_2022_Q3.loc[training_data_2022_Q3['BRAND_NAME'] == 'Wegmans', 'STORE_BRAND_IND'] = 1
training_data_2022_Q3.loc[training_data_2022_Q3['BRAND_NAME'] != 'Wegmans', 'STORE_BRAND_IND'] = 0

In [41]:
training_data_2022_Q3.shape

(144664, 15)

### 10. Add 'Seasonal_IND' 

In [42]:
# add ITEM_STATUS_DESC column to training_data
training_data_2022_Q3 = pd.merge(training_data_2022_Q3, items[['ITEM_NBR', 'ITEM_STATUS_DESC']], on='ITEM_NBR')

# set SEASONAL_IND = 1, if ITEM_STATUS_DESC = Seasonally Suspended, else = 0
training_data_2022_Q3.loc[training_data_2022_Q3['ITEM_STATUS_DESC'] == 'Store Active', 'SEASONAL_IND'] = 0
training_data_2022_Q3.loc[training_data_2022_Q3['ITEM_STATUS_DESC'] == 'New', 'SEASONAL_IND'] = 0
training_data_2022_Q3.loc[training_data_2022_Q3['ITEM_STATUS_DESC'] == 'Seasonally Suspended', 'SEASONAL_IND'] = 1


In [43]:
training_data_2022_Q3.shape

(144664, 17)

### 10. Add 'AVG_MARKUP_PERC'

In [50]:
                 
# add AVG_PERC_MARKUP_2022_Q3 column to training_data
training_data_2022_Q3 = pd.merge(training_data_2022_Q3, pricing[['ITEM_NBR', 'AVG_INSTACART_PERC_MARKUP_2022_7', 'AVG_INSTACART_PERC_MARKUP_2022_8', 'AVG_INSTACART_PERC_MARKUP_2022_9']] , on ='ITEM_NBR', how='left')
training_data_2022_Q3.shape

(144664, 20)

In [51]:
# if an item has no ORIGINALLY_ORDERED_COUNT, then set ORIGINALLY_ORDERED_COUNT = 0
training_data_2022_Q3['AVG_INSTACART_PERC_MARKUP_2022_7'] = training_data_2022_Q3['AVG_INSTACART_PERC_MARKUP_2022_7'].replace(np.nan, 0)
training_data_2022_Q3['AVG_INSTACART_PERC_MARKUP_2022_8'] = training_data_2022_Q3['AVG_INSTACART_PERC_MARKUP_2022_8'].replace(np.nan, 0)
training_data_2022_Q3['AVG_INSTACART_PERC_MARKUP_2022_9'] = training_data_2022_Q3['AVG_INSTACART_PERC_MARKUP_2022_9'].replace(np.nan, 0)

training_data_2022_Q3.shape

(144664, 20)

In [52]:
training_data_2022_Q3['AVG_PERC_MARKUP_2022_Q3'] = (training_data_2022_Q3['AVG_INSTACART_PERC_MARKUP_2022_7'] + training_data_2022_Q3[ 'AVG_INSTACART_PERC_MARKUP_2022_8'] + training_data_2022_Q3['AVG_INSTACART_PERC_MARKUP_2022_9'])/3


In [53]:
#it goes through column A, selects where it's negative & replaces with 0, or if it's not negative it leaves it as is
training_data_2022_Q3.AVG_PERC_MARKUP_2022_Q3 = np.where(training_data_2022_Q3.AVG_PERC_MARKUP_2022_Q3 < 0, 0, training_data_2022_Q3.AVG_PERC_MARKUP_2022_Q3)


In [54]:
training_data_2022_Q3.shape

(144664, 21)

## 11. Add 'PR_SCORE_2022' (Page Rank) Feature 
by Category Rank 
from the graph analysis, where sales rank is the page score of each node in the graph.

In [55]:
graph_df = pd.read_csv('new_train_2021.csv')
graph_df.drop(graph_df.columns[graph_df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
#graph_df.head()


In [56]:
training_data_2022_Q3 = pd.merge(training_data_2022_Q3, graph_df[['ITEM_NBR','PR_SCORE_2021']], on='ITEM_NBR', how = 'left')
training_data_2022_Q3.shape

(144664, 22)

In [57]:
training_data_2022_Q3 = training_data_2022_Q3.rename(columns={'PR_SCORE_2021': 'PR_SCORE'})


## CREATE TARGET VARIABLE

In [58]:
training_data_2022_Q3 = pd.merge(training_data_2022_Q3, training_df[['ITEM_NBR', 'TOP_1000_FASTPICK_ITEM']], on='ITEM_NBR', how='left')
training_data_2022_Q3.shape

(144664, 23)

In [59]:
training_data_2022_Q3['TOP_1000_FASTPICK_ITEM'] = training_data_2022_Q3['TOP_1000_FASTPICK_ITEM'].replace(np.nan, 0)


In [60]:
# for now: add IS_CURRENT_WOODMORE_FASTPICK_FLG as target_column
training_data_2022_Q3 = pd.merge(training_data_2022_Q3, items[['ITEM_NBR', 'IS_CURRENT_WOODMORE_FASTPICK_FLG']], on='ITEM_NBR')
training_data_2022_Q3.shape

(144664, 24)

In [61]:
training_data_2022_Q3 = training_data_2022_Q3.drop_duplicates(subset=['ITEM_NBR'])
training_data_2022_Q3.shape

(122898, 24)

## Save training_data to csv file

In [62]:
training_data_2022_Q3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122898 entries, 0 to 144663
Data columns (total 24 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   ITEM_NBR                          122898 non-null  float64
 1   ORGANIC_IND                       122898 non-null  int64  
 2   BRAND_NAME                        122898 non-null  object 
 3   DEPARTMENT_NAME                   122898 non-null  object 
 4   CATEGORY_NAME                     122898 non-null  object 
 5   CLASS_NAME                        122898 non-null  object 
 6   ONLINE_UNITS_SOLD_2022_Q3         122898 non-null  float64
 7   AGGREGATE_UNITS_SOLD_2022_Q3      122898 non-null  float64
 8   DEPARTMENT_SALES_PERC             122898 non-null  float64
 9   ITEM_SALES_RANK_IN_DEPARTMENT     122898 non-null  float64
 10  CATEGORY_SALES_PERC               122898 non-null  float64
 11  CLASS_SALES_PERC                  122898 non-null  f

In [63]:
# Keep only numeric attributes for modeling 
training_x = training_data_2022_Q3.drop(columns=['BRAND_NAME','CLASS_NAME','CATEGORY_NAME', 'DEPARTMENT_NAME','ITEM_STATUS_DESC', 'AVG_INSTACART_PERC_MARKUP_2022_7', 'AVG_INSTACART_PERC_MARKUP_2022_8', 'AVG_INSTACART_PERC_MARKUP_2022_9'])


In [64]:
training_x.shape

(122898, 16)

In [65]:
# Save file 
training_x.to_csv('training_data_2022_Q3.csv', index=False)

## END of Feature Engineering file